# 6. Grouping, joining, and sorting

## Exercise 29 - Longest taxi rides

In [311]:
# read in CSV, specify columns
jan_19 = pd.read_csv(root_path + 'nyc_taxi_2019-01.csv',
                     usecols=['passenger_count', 'trip_distance', 'total_amount'])
jan_19

,passenger_count,trip_distance,total_amount
0,1,1.50,9.95
1,1,2.60,16.30
2,3,0.00,5.80
3,5,0.00,7.55
4,5,0.00,55.55
...,...,...,...
7667787,1,4.79,23.16
7667788,1,0.00,0.00
7667789,1,0.00,0.00
7667790,1,0.00,0.00


In [312]:
# descending avg cost of 20 longest (distance) taxi rides
jan_19.sort_values('trip_distance', ascending=False).head(20).loc[:,'total_amount'].mean()

290.00999999999993

In [313]:
# ascending avg cost of 20 longest (distance) taxi rides
jan_19.sort_values('trip_distance').tail(20)['total_amount'].mean()

290.01000000000005

In [314]:
# sort by ascending passenger count and descending trip distance, find avg price top 50 rides
jan_19.sort_values(['passenger_count', 'trip_distance'], ascending=[True, False]).head(50)['total_amount'].mean()

135.49739999999997

### Exercise 29b

In [316]:
# which 5 rides did passengers pay the most per mile? how far did they go?
jan_19['ppm'] = jan_19['total_amount'] / jan_19['trip_distance']
jan_19[jan_19['trip_distance'] != 0.00].sort_values('ppm', ascending=False).head()

,passenger_count,trip_distance,total_amount,ppm
2499600,1,2.40,"623,261.66","259,692.36"
478791,1,0.10,"6,667.45","66,674.50"
7099014,4,0.01,415.30,"41,530.00"
6403254,1,0.01,322.30,"32,230.00"
4136499,1,0.01,273.96,"27,396.00"


In [317]:
# which 10 multipassenger rides did each individual pay the greatest amount?
jan_19['ppp'] = jan_19['total_amount'] / jan_19['passenger_count']
jan_19[jan_19['passenger_count'] > 1].sort_values('ppp', ascending=False).head(10)

,passenger_count,trip_distance,total_amount,ppm,ppp
2972145,2,19.90,589.96,29.65,294.98
3014027,2,16.60,560.76,33.78,280.38
3842620,2,110.04,515.82,4.69,257.91
7593395,2,83.61,449.32,5.37,224.66
149362,2,17.20,426.80,24.81,213.40
5726185,2,65.05,416.82,6.41,208.41
6857368,2,0.00,411.36,inf,205.68
6496403,2,0.00,410.95,inf,205.47
4751745,2,100.78,403.50,4.00,201.75
1154626,2,0.00,400.80,inf,200.40


In [318]:
# use ignore_index with loc to get mean total_amount for 20 longest trips
jan_19.sort_values('trip_distance', ascending=False, ignore_index=True).loc[:20]['total_amount'].mean()

300.76285714285706

## Exercise 30 - Taxi ride comparison

In [320]:
# load CSV again
jan_19 = pd.read_csv(root_path + 'nyc_taxi_2019-01.csv',
                     usecols=['passenger_count', 'trip_distance', 'total_amount'])
jan_19

,passenger_count,trip_distance,total_amount
0,1,1.50,9.95
1,1,2.60,16.30
2,3,0.00,5.80
3,5,0.00,7.55
4,5,0.00,55.55
...,...,...,...
7667787,1,4.79,23.16
7667788,1,0.00,0.00
7667789,1,0.00,0.00
7667790,1,0.00,0.00


In [321]:
# find mean cost for each passenger count, sort by lowest to highest
jan_19.groupby('passenger_count')['total_amount'].mean().sort_values()

passenger_count
6   15.44
5   15.55
3   15.60
1   15.61
4   15.65
2   15.83
0   18.66
9   31.09
7   48.28
8   64.11
Name: total_amount, dtype: float64

In [322]:
# sort again by increasing number of passengers
jan_19.groupby('passenger_count')['total_amount'].mean().sort_index()

passenger_count
0   18.66
1   15.61
2   15.83
3   15.60
4   15.65
5   15.55
6   15.44
7   48.28
8   64.11
9   31.09
Name: total_amount, dtype: float64

In [323]:
# create new column, 'trip_distance_group'
# once again can't use cut because of irregular bin specifications

# categorize trip distances into 3 lengths
def categorize(x):
    if x < 2:
        return 'short'
    if x >= 2 and x <=10:
        return 'medium'
    if x > 10:
        return 'long'

# apply to trip_distance column
jan_19['trip_distance_group'] = jan_19['trip_distance'].apply(categorize)

# view counts of each category
jan_19['trip_distance_group'].value_counts(dropna=False)

trip_distance_group
short     4749484
medium    2494589
long       423719
Name: count, dtype: int64

In [324]:
# average passengers per trip length category sorted highest to lowest
jan_19.groupby('trip_distance_group')['passenger_count'].mean().sort_values(ascending=False)

trip_distance_group
long     1.59
medium   1.58
short    1.56
Name: passenger_count, dtype: float64

### Exercise 30b

In [326]:
# create single df rides from Jan '19 and Jan '20 w/ column year indicating which year ride comes from

jan_19 = pd.read_csv(root_path + 'nyc_taxi_2019-01.csv',
                     usecols=['passenger_count', 'trip_distance', 'total_amount'])
jan_19['year'] = 2019

jan_20 = pd.read_csv(root_path + 'nyc_taxi_2020-01.csv',
                     usecols=['passenger_count', 'trip_distance', 'total_amount'])
jan_20['year'] = 2020

jan_rides = pd.concat([jan_19, jan_20])
jan_rides

,passenger_count,trip_distance,total_amount,year
0,1.00,1.50,9.95,2019
1,1.00,2.60,16.30,2019
2,3.00,0.00,5.80,2019
3,5.00,0.00,7.55,2019
4,5.00,0.00,55.55,2019
...,...,...,...,...
6405003,NaN,3.24,21.14,2020
6405004,NaN,22.13,62.46,2020
6405005,NaN,10.51,51.90,2020
6405006,NaN,5.49,30.22,2020


In [327]:
# use groupby to compare average cost of taxi in January of each year
jan_rides.groupby('year')['total_amount'].mean()

year
2019   15.68
2020   18.66
Name: total_amount, dtype: float64

In [328]:
# create a two-level grouping, by year then passenger_count
jan_rides.groupby(['year', 'passenger_count'])['total_amount'].mean()

year  passenger_count
2019  0.00              18.66
      1.00              15.61
      2.00              15.83
      3.00              15.60
      4.00              15.65
      5.00              15.55
      6.00              15.44
      7.00              48.28
      8.00              64.11
      9.00              31.09
2020  0.00              18.06
      1.00              18.34
      2.00              19.05
      3.00              18.74
      4.00              19.13
      5.00              18.23
      6.00              18.37
      7.00              71.14
      8.00              58.20
      9.00              81.24
Name: total_amount, dtype: float64

In [329]:
# use corr and sort_values to find highest correlation
jan_rides.corr().sort_values('passenger_count')

,passenger_count,trip_distance,total_amount,year
year,-0.02,0.00,0.01,1.00
total_amount,-0.00,0.00,1.00,0.01
trip_distance,0.01,1.00,0.00,0.00
passenger_count,1.00,0.01,-0.00,-0.02


## Exercise 31 - Tourist spending per country

In [331]:
# load in CSV, specify columns
tourism = pd.read_csv(root_path + 'oecd_tourism.csv',
                      usecols='LOCATION SUBJECT TIME Value'.split())
tourism

,LOCATION,SUBJECT,TIME,Value
0,AUS,INT_REC,2008,"31,159.80"
1,AUS,INT_REC,2009,"29,980.70"
2,AUS,INT_REC,2010,"35,165.50"
3,AUS,INT_REC,2011,"38,710.10"
4,AUS,INT_REC,2012,"38,003.70"
...,...,...,...,...
1229,SRB,INT-EXP,2015,"1,253.64"
1230,SRB,INT-EXP,2016,"1,351.10"
1231,SRB,INT-EXP,2017,"1,549.18"
1232,SRB,INT-EXP,2018,"1,837.32"


In [332]:
# 5 countries receiving most tourism dollars, on average, across years in dataset
tourism[tourism['SUBJECT'] == 'INT_REC'].groupby('LOCATION')['Value'].mean().sort_values(ascending=False).head()

LOCATION
USA   201,613.50
ESP    69,655.82
FRA    65,063.34
DEU    53,408.57
GBR    51,752.09
Name: Value, dtype: float64

In [333]:
# 5 countries whose citizens spent the least amount of tourist dollars, on average, across years
tourism[tourism['SUBJECT'] == 'INT-EXP'].groupby('LOCATION')['Value'].mean().sort_values(ascending=True).head()

LOCATION
MLT     387.80
CRI     867.07
LVA     919.55
ISL   1,072.82
HRV   1,115.63
Name: Value, dtype: float64

In [334]:
# load in oecd_locations, using abbreviated location name as index
locations = pd.read_csv(root_path + 'oecd_locations.csv',
                       names=['abbreviated', 'full'],
                       index_col='abbreviated')
locations.head()

,full
abbreviated,
AUS,Australia
AUT,Austria
BEL,Belgium
CAN,Canada
DNK,Denmark


In [335]:
# join both dataframes, without a 'LOCATION' column but a 'name' column with full name of country

# set LOCATION as index
tourism = tourism.set_index('LOCATION')

# join together
tourism_name = tourism.join(locations)
tourism_name.head()

,SUBJECT,TIME,Value,full
LOCATION,,,,
AUS,INT_REC,2008,"31,159.80",Australia
AUS,INT_REC,2009,"29,980.70",Australia
AUS,INT_REC,2010,"35,165.50",Australia
AUS,INT_REC,2011,"38,710.10",Australia
AUS,INT_REC,2012,"38,003.70",Australia


In [336]:
# finalize by dropping index and renaming country column
tourism_name.reset_index(inplace=True, drop=True)
tourism_name = tourism_name.rename(columns={'full': 'name'})

In [337]:
# rerun queries from steps 2 and 3, returning full country name rather than abbreviation

# 5 countries receiving most tourism dollars, on average, across years in dataset
tourism_name[tourism_name['SUBJECT'] == 'INT_REC'].groupby('name')['Value'].mean().sort_values(ascending=False).head()

name
United States    201,613.50
France            65,063.34
Germany           53,408.57
United Kingdom    51,752.09
Italy             44,930.21
Name: Value, dtype: float64

In [338]:
# 5 countries whose citizens spent the least amount of tourist dollars, on average, across years
tourism_name[tourism_name['SUBJECT'] == 'INT-EXP'].groupby('name')['Value'].mean().sort_values(ascending=True).head()

name
Hungary    2,918.39
Finland    5,877.08
Israel     6,726.52
Denmark   11,326.17
Austria   11,934.56
Name: Value, dtype: float64

In [339]:
# output is not the same, as not every abbreviated country had a "full name" in the locations df
# resulted in many null values for "name"

### Exercise 31b

In [341]:
# join order does not matter

In [342]:
# mean tourism income per year rather than country, inspect Great Recession starting in 2008
tourism[tourism['SUBJECT'] == 'INT_REC'].groupby('TIME')['Value'].mean()

TIME
2008   16,841.15
2009   15,081.29
2010   16,003.94
2011   17,788.74
2012   18,216.11
2013   19,296.54
2014   20,198.82
2015   19,301.87
2016   19,574.94
2017   20,763.39
2018   22,436.34
2019   23,005.94
Name: Value, dtype: float64

In [343]:
# reset index on locations_df
locations = locations.reset_index()

In [344]:
# rename columns to match
locations = locations.rename(columns={'abbreviated': 'LOCATION', 'full': 'name'})

In [345]:
# run the join, specify location column to be used on the caller instead of index (argument df will always join on index)
locations.join(tourism, on='LOCATION').set_index('LOCATION')

,name,SUBJECT,TIME,Value
LOCATION,,,,
AUS,Australia,INT_REC,2008,"31,159.80"
AUS,Australia,INT_REC,2009,"29,980.70"
AUS,Australia,INT_REC,2010,"35,165.50"
AUS,Australia,INT_REC,2011,"38,710.10"
AUS,Australia,INT_REC,2012,"38,003.70"
...,...,...,...,...
ISR,Israel,INT-EXP,2015,"7,507.00"
ISR,Israel,INT-EXP,2016,"8,210.30"
ISR,Israel,INT-EXP,2017,"8,986.00"


## Chapter 6 Notes
- When using join method, argument df always joins on index, even when specifying a different joining criteria for initial df (on='col_name')
- sort_values(ignore_index=True) resets the index after sorting